# dataframe with transfermarket

## importing clean_data

In [1]:
!pip install ipynb
!pip install aiohttp

In [2]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import asyncio
import aiohttp
from aiohttp.client import ClientSession
import nest_asyncio
import string
import re
nest_asyncio.apply()

In [3]:
df = pd.read_csv("../raw_data/clean_data.csv")

### Korean translation didn't work

In [ ]:
kor_df = df[df.nat == 'KOR']

In [ ]:
kor_df.name[:200]

In [ ]:
from googletrans import Translator

In [ ]:
def translate_to_korean(english_names):
    translator = Translator()
    korean_names = [translator.translate(name, dest='ko').text for name in english_names.replace("-"," ")]
    return pd.Series(korean_names)

In [ ]:
korean_names_series = translate_to_korean(kor_df.name[:5])

In [ ]:
korean_names_series

In [ ]:
def translate_to_korean(name):
    translator = Translator()
    translation = translator.translate(name, dest='ko')
    korean_name = translation.text
    return korean_name
 Example usage:
english_name = 'Son Heung-Min'
korean_name = translate_to_korean(english_name)

In [ ]:
korean_name

### other

%%time
async def fetch_and_process(session, url, name):
    async with session.get(url, headers=headers) as response:
        content = await response.text()
        soup = BeautifulSoup(content, 'html.parser')
        def player_link(): 
            if soup.find(class_="hauptlink"):
                return "https://www.transfermarkt.com" + soup.find(class_="hauptlink").a.get('href')
            return "player not found"
            
        return name, player_link()

async def main(names_url):
    my_conn = aiohttp.TCPConnector(limit=10)
    
    async with ClientSession(connector=my_conn) as session:
        tasks = [fetch_and_process(session, url, name) for name, url in zip(names, names_url)]
        results = await asyncio.gather(*tasks)
    
    # Process the results, which are tuples of (name, player_link)
    player_link = {name: link for name, link in results}
    return player_link
    
    # Process the results, which are tuples of (name, player_link)
    player_link = {name: link for name, link in results}
    await player_link

player_link = asyncio.run(main(names_url[0]))

## our base URL

In [4]:
url = "https://www.transfermarkt.com/schnellsuche/ergebnis/schnellsuche?query="
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/109.0.0.0 Safari/537.36',
}

In [22]:
names = pd.Series(df.name)[:8750]

In [23]:
%%time
names_url = names.map(lambda x: url + x.lower().replace(" ", "+"))
names_url

CPU times: user 8.75 ms, sys: 3.41 ms, total: 12.2 ms
Wall time: 11.9 ms


0       https://www.transfermarkt.com/schnellsuche/erg...
1       https://www.transfermarkt.com/schnellsuche/erg...
2       https://www.transfermarkt.com/schnellsuche/erg...
3       https://www.transfermarkt.com/schnellsuche/erg...
4       https://www.transfermarkt.com/schnellsuche/erg...
                              ...                        
8745    https://www.transfermarkt.com/schnellsuche/erg...
8746    https://www.transfermarkt.com/schnellsuche/erg...
8747    https://www.transfermarkt.com/schnellsuche/erg...
8748    https://www.transfermarkt.com/schnellsuche/erg...
8749    https://www.transfermarkt.com/schnellsuche/erg...
Name: name, Length: 8750, dtype: object

### first function

%%time
async def fetch(session, url, headers):
    async with session.get(url, headers=headers) as response:
        return await response.text()

async def scrape_page(url, headers, club_selector, image_selector):
    async with aiohttp.ClientSession() as session:
        html = await fetch(session, url, headers)
        soup = BeautifulSoup(html, 'html.parser')

        # Extracting club
        club_element = soup.select_one(club_selector)
        club = club_element['alt'] if club_element else None

        # Extracting profile image
        image_element = soup.select_one(image_selector)
        profile_image = image_element['src'] if image_element else None

        return {'url': url, 'club': club, 'profile_image': profile_image}

async def main(names_url):
    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/109.0.0.0 Safari/537.36',
    }

    # Specify the CSS selectors for the club and image elements
    club_selector = 'a[title^=""] img.tiny_wappen'
    image_selector = 'img.bilderrahmen-fixed'

    # Use asyncio.gather to concurrently scrape multiple pages
    tasks = [scrape_page(url, headers, club_selector, image_selector) for url in names_url]
    results = await asyncio.gather(*tasks)

    for result in results:
        print(result)

if __name__ == "__main__":
    # Example Pandas Series with URLs
    names_url = names_url

    asyncio.run(main(names_url))

### second function

async def fetch(session, url, headers, semaphore):
    async with semaphore:
        async with session.get(url, headers=headers) as response:
            return await response.text()

async def scrape_page(url, headers, club_selector, image_selector, semaphore):
    async with aiohttp.ClientSession() as session:
        html = await fetch(session, url, headers, semaphore)
        soup = BeautifulSoup(html, 'html.parser')

        # Extracting club
        club_element = soup.select_one(club_selector)
        club = club_element['alt'] if club_element else 'None'

        # Extracting profile image
        image_element = soup.select_one(image_selector)
        profile_image = image_element['src'] if image_element else 'None'

        return {'url': url, 'club': club, 'profile_image': profile_image}

async def main(names_url):
    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/109.0.0.0 Safari/537.36',
    }

    # Specify the CSS selectors for the club and image elements
    club_selector = 'a[title^=""] img.tiny_wappen'
    image_selector = 'img.bilderrahmen-fixed'

    # Set the maximum number of concurrent requests
    max_concurrent_requests = 4000
    semaphore = asyncio.Semaphore(max_concurrent_requests)

    # Use asyncio.gather to concurrently scrape multiple pages
    tasks = [scrape_page(url, headers, club_selector, image_selector, semaphore) for url in names_url]
    results = await asyncio.gather(*tasks)

    # Convert the results list to a DataFrame
    df = pd.DataFrame(results)

    return df

if __name__ == "__main__":
    # Example Pandas Series with URLs
    names_url = names_url

    import time
    start_time = time.time()

    result_df = asyncio.run(main(names_url))

    end_time = time.time()

    print(f"Total time taken: {end_time - start_time} seconds")
result_df

result_df.head(20)

result_df[result_df.club == 'None']

result_df.info()

### second function upgrade

In [28]:
async def fetch(session, url, headers, semaphore):
    async with semaphore:
        async with session.get(url, headers=headers) as response:
            return await response.text()

async def scrape_page(url, headers, club_selector, image_selector, year_selector, market_value_selector, name_selector, semaphore):
    async with aiohttp.ClientSession() as session:
        html = await fetch(session, url, headers, semaphore)
        soup = BeautifulSoup(html, 'html.parser')

        # Extracting club
        club_element = soup.select_one(club_selector)
        club = club_element['alt'] if club_element else 'None'

        # Extracting profile image
        image_element = soup.select_one(image_selector)
        profile_image = image_element['src'] if image_element else 'None'

        # Extracting year
        year_element = soup.select_one(year_selector)
        year = year_element.text.strip() if year_element else 'None'

        # Extracting market value
        market_value_element = soup.select_one(market_value_selector)
        market_value = market_value_element.text.strip() if market_value_element else 'None'
        
        # Extracting name
        name_element = soup.select_one(name_selector)
        name = name_element.text.strip() if name_element else 'None'

        return {'name': name, 'club': club, 'profile_image': profile_image, 'year': year, 'market_value': market_value}

async def main(names_url):
    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/109.0.0.0 Safari/537.36',
    }

    # Specify the CSS selectors for the club, image, year, and market value elements
    club_selector = 'a[title^=""] img.tiny_wappen'
    image_selector = 'img.bilderrahmen-fixed'
    year_selector = 'td.zentriert:nth-of-type(4)'  # Update with the correct selector for the year
    market_value_selector = 'td.rechts.hauptlink'  # Update with the correct selector for market value
    name_selector = 'td.hauptlink a'

    # Set the maximum number of concurrent requests
    max_concurrent_requests = 3000
    semaphore = asyncio.Semaphore(max_concurrent_requests)

    # Use asyncio.gather to concurrently scrape multiple pages
    tasks = [scrape_page(url, headers, club_selector, image_selector, year_selector, market_value_selector, name_selector, semaphore) for url in names_url]
    results = await asyncio.gather(*tasks)

    # Convert the results list to a DataFrame
    df = pd.DataFrame(results)

    return df

if __name__ == "__main__":
    # Example Pandas Series with URLs
    names_url = names_url

    import time
    start_time = time.time()

    result_df = asyncio.run(main(names_url))

    end_time = time.time()

    print(f"Total time taken: {end_time - start_time} seconds")
result_df

### new version

In [26]:
import asyncio
import aiohttp
from bs4 import BeautifulSoup
import pandas as pd

async def fetch(session, url, headers, semaphore):
    async with semaphore:
        try:
            async with session.get(url, headers=headers) as response:
                return await response.text()
        except Exception as e:
            print(f"Error fetching {url}: {e}")
            return None

async def scrape_page(url, headers, club_selector, image_selector, year_selector, market_value_selector, name_selector, semaphore):
    async with aiohttp.ClientSession() as session:
        html = await fetch(session, url, headers, semaphore)
        if html is None:
            return None

        soup = BeautifulSoup(html, 'html.parser')

        # Extracting club
        club_element = soup.select_one(club_selector)
        club = club_element['alt'] if club_element else 'None'

        # Extracting profile image
        image_element = soup.select_one(image_selector)
        profile_image = image_element['src'] if image_element else 'None'

        # Extracting year
        year_element = soup.select_one(year_selector)
        year = year_element.text.strip() if year_element else 'None'

        # Extracting market value
        market_value_element = soup.select_one(market_value_selector)
        market_value = market_value_element.text.strip() if market_value_element else 'None'

        # Extracting name
        name_element = soup.select_one(name_selector)
        name = name_element.text.strip() if name_element else 'None'

        return {'name': name, 'club': club, 'profile_image': profile_image, 'year': year, 'market_value': market_value}

async def main(names_url):
    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/109.0.0.0 Safari/537.36',
    }

    # Specify the CSS selectors for the club, image, year, and market value elements
    club_selector = 'a[title^=""] img.tiny_wappen'
    image_selector = 'img.bilderrahmen-fixed'
    year_selector = 'td.zentriert:nth-of-type(4)'
    market_value_selector = 'td.rechts.hauptlink.vereinsfarben'
    name_selector = 'td.hauptlink a.spielprofil_tooltip'

    # Set the maximum number of concurrent requests
    max_concurrent_requests = 3000
    semaphore = asyncio.Semaphore(max_concurrent_requests)

    # Use asyncio.gather to concurrently scrape multiple pages
    tasks = [scrape_page(url, headers, club_selector, image_selector, year_selector, market_value_selector, name_selector, semaphore) for url in names_url]
    results = await asyncio.gather(*tasks)

    # Convert the results list to a DataFrame
    df = pd.DataFrame(results)

    return df

if __name__ == "__main__":
    # Example Pandas Series with URLs
    names_url = names_url

    import time
    start_time = time.time()

    result_df = asyncio.run(main(names_url))

    end_time = time.time()

    print(f"Total time taken: {end_time - start_time} seconds")
result_df

### scrape club logos

In [ ]:
clubs = pd.Series(df.club.unique())

In [ ]:
clubs_url = clubs.map(lambda x: url + x.lower().replace(" ", "+"))
clubs_url

In [ ]:
async def scrape_page(url, headers, club_logo_selector, club_name_selector, semaphore):
    async with aiohttp.ClientSession() as session:
        html = await fetch(session, url, headers, semaphore)
        soup = BeautifulSoup(html, 'html.parser')

        # Extracting club logo
        club_logo_element = soup.select_one(club_logo_selector)
        club_logo = club_logo_element['src'] if club_logo_element else 'None'

        # Extracting club name
        club_name_element = soup.select_one(club_name_selector)
        club_name = club_name_element['alt'].strip() if club_name_element and 'alt' in club_name_element.attrs else 'None'

        return {'club_name': club_name, 'club_logo': club_logo}

async def main(names_url):
    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/109.0.0.0 Safari/537.36',
    }

    # Specify the CSS selectors for the club logo and club name elements
    club_logo_selector = 'td.zentriert.suche-vereinswappen img'
    club_name_selector = 'td.zentriert.suche-vereinswappen img'


    # Set the maximum number of concurrent requests
    max_concurrent_requests = 4000
    semaphore = asyncio.Semaphore(max_concurrent_requests)

    # Use asyncio.gather to concurrently scrape multiple pages
    tasks = [scrape_page(url, headers, club_logo_selector, club_name_selector, semaphore) for url in names_url]
    results = await asyncio.gather(*tasks)

    # Convert the results list to a DataFrame
    df = pd.DataFrame(results)

    return df

if __name__ == "__main__":
    # Example Pandas Series with URLs
    names_url = clubs_url

    import time
    start_time = time.time()

    result_df = asyncio.run(main(names_url))

    end_time = time.time()

    print(f"Total time taken: {end_time - start_time} seconds")
result_df.head(50)

In [ ]:
result_df["fm_name"] = clubs

In [ ]:
result_df

In [ ]:
result_df[result_df.club_name == 'None']

## rest

In [ ]:
def common_names(dictio):
    num = 0
    result = []
    for x in dictio.keys():
        if not " " in x:
            result.append(x)
            num += 1
        result = result
    print(num)
    return result
check_names = common_names(player_link)
check_names

In [ ]:
allowed_chars = string.ascii_letters

In [ ]:
def name_filter(name):
    for x in name:
        if x in allowed_chars:
            continue
        return False
    return True

In [ ]:
def clean_player_info(player_info):
    cleaned_info = []
    
    if player_info is str:
        return player_info
    else:

        # Extract name
        cleaned_info.append(player_info[0])
        

        # Extract age
        try:
            if len(player_info) < 11:
                cleaned_info.append(player_info[2])
            cleaned_info.append(player_info[3])
        except: 
            return cleaned_info
            

        '''# Extract height
        try:
            cleaned_info.append(player_info[4].replace("\xa0m", "").replace(",", "."))
        except:
            return cleaned_info
        # Extract position
        try:
            cleaned_info.append(player_info[6].strip())
        except:
            cleaned_info.append(player_info[-1].strip())'''

    return cleaned_info

I have some issues with cleaning the data of Dutch players and probably some other countries but I didn't notice them yet

In [ ]:
async def process(session, url):
    try:
        async with session.get(url, headers=headers) as response:
            response.raise_for_status()
            content = await response.text()
            soup = BeautifulSoup(content, 'html.parser')

            # Extract the full name if found
            full_name = "Full Name not found"
            info_content = soup.find_all(class_="info-table__content info-table__content--bold")
            if info_content:
                full_name = [info.text for info in info_content]
                

            return full_name
    except Exception as e:
        print(f"Error processing {url}: {e}")
        return "Error"

async def info(player_links):
    my_conn = aiohttp.TCPConnector(limit=10)

    async with ClientSession(connector=my_conn) as session:
        # Create a list of tasks for processing player information
        tasks = [process(session, url) for url in player_links.values()]

        # Execute the tasks concurrently and collect the results
        full_names = await asyncio.gather(*tasks)

    # Combine player names with the keys from the original player_links dictionary
    player_info = {name: full_name for name, full_name in zip(player_links.keys(), full_names)}
    player_info = {name: ([name] + full_name  if ", " in full_name[0] else full_name) for name, full_name in player_info.items()}
    #player_info = {name: clean_player_info(full_name) for name, full_name in player_info.items()}
    
    

    return player_info



full_name = asyncio.run(info(player_link))
full_name

In [ ]:
print(len(full_name['Jaden Slory']))
full_name['Jaden Slory']

In [ ]:
print(len(full_name['Maximilian Beier']))
full_name['Maximilian Beier']

In [ ]:
print(len(full_name['Eliesse Ben Seghir']))
full_name['Eliesse Ben Seghir']

In [ ]:
print(len(full_name['Pedri']))
full_name['Pedri']

In [ ]:
full_name['Joris Chotard']
 'Ayman Kari': 'Ayman Kari',
 'Mohamed El Arouch': 'Mohamed El Arouch',
 'Malo Gusto': 'Malo Gusto',
 'Lucas Gourna-Douath': 'Lucas Gourna-Douath',
 'Koba Lein': 'Full Name not found',

In [ ]:
a = [2,3,4,5]
[1] + a[1:]

In [ ]:
'PedroGonzálezLópez'.isalpha()

In [ ]:

def clean_player_info(player_info):
    cleaned_info = []

    # Extract name
    cleaned_info.append(player_info[0])
        

    # Extract age
    cleaned_info.append(player_info[3])

    # Extract height
    cleaned_info.append(player_info[4].replace("\xa0m", "").replace(",", "."))

    # Extract position
    cleaned_info.append(player_info[6].strip())

    return cleaned_info

clean_player_info(full_name['Pedri'])
#clean_player_info(full_name['Joris Chotard'])

In [ ]:
full_name['Pedri']

In [ ]:
def link_filter(link, name):
    if name in check_names:
        if f'/{name.lower()}/' in link:
            return link
        return 'confusing_name'
    return link
player_link = {key: link_filter(value, key) for key, value in player_link.items()}
player_link

In [ ]:
def num_of_nones(dictio):
    result = 0
    for x,y in dictio.items():
        if y == 'player not found':
            result += 1
            print(x)
        result = result
    return result

✅ for common_names
if /common_name/ is not in link
return confusing_name

from every transfermarket link

we will retrieve:

✅ the full name --> will help us for the link on sorare

current club --> FM is outdated

current market value

position

age

---
make a dictionary with the available leagues on sorare

create two tables with:

available vs non_available on sorare

filter the available on injury